In [ ]:
!pip install gdown lightning

In [ ]:
!gdown https://drive.google.com/uc?id=1rh_21CJliIkuahqqaWETH8Zf06qEA7eG

In [ ]:
!mkdir temp
!mkdir data
!unzip -q data.zip -d temp
!mv temp/data/* data/
!rm -rf temp

In [ ]:
!ls -l data

In [ ]:
from pathlib import Path
from typing import List, Optional, Tuple

import cv2
import lightning.pytorch as pl
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch import LightningDataModule, LightningModule, Trainer
from lightning.pytorch.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from lightning.pytorch.loggers import TensorBoardLogger
from PIL import Image
from scipy import ndimage
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision import models, transforms

In [ ]:
# ============================================================================
# Configuration
# ============================================================================
class Config:
    # Data paths
    DATA_DIR = "./data"
    TRAIN_DATA_DIR = "./data/train_data"
    TEST_DATA_DIR = "./data/test_data"
    TRAIN_LABELS_PATH = "./data/train_labels_cleaned.csv"
    OUTPUT_PATH = "./predictions.csv"

    # Class labels
    CLASSES = ["Luminal A", "Luminal B", "HER2(+)", "Triple negative"]
    NUM_CLASSES = 4

    # Image settings
    IMG_SIZE = 512  # Larger size for histopathology
    USE_MASK = True

    # Mask mode: "multiply" (apply mask), "channel" (4th channel), "attention" (learned attention)
    MASK_MODE = "channel"  # Use mask as 4th input channel
    IN_CHANNELS = 4  # 3 (RGB) + 1 (mask) when MASK_MODE="channel", else 3

    # Tissue detection settings
    TISSUE_THRESHOLD = 0.8  # Threshold for tissue detection (lower = more sensitive)
    MIN_TISSUE_AREA = 0.05  # Minimum tissue area ratio
    PADDING = 50  # Padding around tissue bounding box

    # Patch-based settings (for very large images)
    USE_PATCHES = False  # Set to False for simpler 4-channel approach
    PATCH_SIZE = 512
    NUM_PATCHES = 8  # Number of patches to sample per image

    # Stain normalization
    USE_STAIN_NORMALIZATION = True

    # Training settings
    BATCH_SIZE = 4
    NUM_WORKERS = 2
    MAX_EPOCHS = 100
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-4

    # Model settings
    MODEL_NAME = "efficientnet_b3"
    PRETRAINED = True

    # Validation split
    VAL_SPLIT = 0.2
    RANDOM_SEED = 42

In [ ]:
# ============================================================================
# Stain Normalization (Macenko method simplified)
# ============================================================================
class StainNormalizer:
    """Simple stain normalization for H&E images."""

    def __init__(self):
        # Reference stain vectors (standard H&E)
        self.target_means = np.array([148.60, 41.56, 105.97])  # LAB color space
        self.target_stds = np.array([41.56, 9.01, 6.67])

    def normalize(self, img: np.ndarray) -> np.ndarray:
        """Normalize stain colors using LAB color space."""
        if img.dtype != np.uint8:
            img = (img * 255).astype(np.uint8)

        # Convert to LAB
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB).astype(np.float32)

        # Normalize each channel
        for i in range(3):
            channel = lab[:, :, i]
            channel_mean = channel.mean()
            channel_std = channel.std() + 1e-6

            # Normalize to target distribution
            lab[:, :, i] = ((channel - channel_mean) / channel_std) * self.target_stds[
                i
            ] + self.target_means[i]

        # Clip and convert back
        lab = np.clip(lab, 0, 255).astype(np.uint8)
        normalized = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)

        return normalized


class ReinhardNormalizer:
    """Reinhard stain normalization."""

    def __init__(self):
        # Target statistics (can be computed from a reference image)
        self.target_means = None
        self.target_stds = None
        self._set_default_target()

    def _set_default_target(self):
        """Set default target statistics for H&E."""
        self.target_means = np.array([180.0, 135.0, 165.0])
        self.target_stds = np.array([25.0, 15.0, 20.0])

    def fit(self, target_img: np.ndarray):
        """Fit normalizer to a target image."""
        lab = cv2.cvtColor(target_img, cv2.COLOR_RGB2LAB).astype(np.float32)
        self.target_means = np.array([lab[:, :, i].mean() for i in range(3)])
        self.target_stds = np.array([lab[:, :, i].std() for i in range(3)])

    def normalize(self, img: np.ndarray) -> np.ndarray:
        """Normalize image to target statistics."""
        lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB).astype(np.float32)

        for i in range(3):
            src_mean = lab[:, :, i].mean()
            src_std = lab[:, :, i].std() + 1e-6
            lab[:, :, i] = ((lab[:, :, i] - src_mean) / src_std) * self.target_stds[
                i
            ] + self.target_means[i]

        lab = np.clip(lab, 0, 255).astype(np.uint8)
        return cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)


In [ ]:
# ============================================================================
# Tissue Detection and Preprocessing
# ============================================================================
class TissueDetector:
    """Detect and extract tissue regions from histopathology images."""

    def __init__(
        self, threshold: float = 0.8, min_area_ratio: float = 0.05, padding: int = 50
    ):
        self.threshold = threshold
        self.min_area_ratio = min_area_ratio
        self.padding = padding

    def detect_tissue_mask(self, img: np.ndarray) -> np.ndarray:
        """Create binary mask of tissue regions."""
        # Convert to grayscale
        if len(img.shape) == 3:
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        else:
            gray = img

        # Tissue is darker than background (which is usually white/light gray)
        # Use Otsu's thresholding
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Alternative: threshold based on intensity
        # Background is typically > 200 in grayscale
        intensity_mask = gray < 220

        # Combine with saturation (tissue has color, background doesn't)
        if len(img.shape) == 3:
            hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            saturation = hsv[:, :, 1]
            saturation_mask = saturation > 10
            tissue_mask = intensity_mask & saturation_mask
        else:
            tissue_mask = intensity_mask

        # Clean up mask
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        tissue_mask = cv2.morphologyEx(
            tissue_mask.astype(np.uint8), cv2.MORPH_CLOSE, kernel
        )
        tissue_mask = cv2.morphologyEx(tissue_mask, cv2.MORPH_OPEN, kernel)

        # Fill holes
        tissue_mask = ndimage.binary_fill_holes(tissue_mask).astype(np.uint8)

        return tissue_mask * 255

    def get_tissue_bbox(self, mask: np.ndarray) -> Tuple[int, int, int, int]:
        """Get bounding box of tissue region."""
        coords = np.where(mask > 0)
        if len(coords[0]) == 0:
            return 0, 0, mask.shape[1], mask.shape[0]

        y_min, y_max = coords[0].min(), coords[0].max()
        x_min, x_max = coords[1].min(), coords[1].max()

        # Add padding
        h, w = mask.shape[:2]
        y_min = max(0, y_min - self.padding)
        y_max = min(h, y_max + self.padding)
        x_min = max(0, x_min - self.padding)
        x_max = min(w, x_max + self.padding)

        return x_min, y_min, x_max, y_max

    def crop_tissue(
        self, img: np.ndarray, mask: Optional[np.ndarray] = None
    ) -> np.ndarray:
        """Crop image to tissue region."""
        if mask is None:
            mask = self.detect_tissue_mask(img)

        x_min, y_min, x_max, y_max = self.get_tissue_bbox(mask)
        cropped = img[y_min:y_max, x_min:x_max]

        # Ensure minimum size
        if cropped.shape[0] < 100 or cropped.shape[1] < 100:
            return img  # Return original if crop is too small

        return cropped

    def get_tissue_patches(
        self,
        img: np.ndarray,
        patch_size: int = 512,
        num_patches: int = 8,
        mask: Optional[np.ndarray] = None,
    ) -> List[np.ndarray]:
        """Extract patches from tissue regions."""
        if mask is None:
            mask = self.detect_tissue_mask(img)

        h, w = img.shape[:2]
        patches = []

        # Find tissue coordinates
        tissue_coords = np.where(mask > 0)
        if len(tissue_coords[0]) == 0:
            # No tissue found, return center crop
            cy, cx = h // 2, w // 2
            half = patch_size // 2
            patch = img[
                max(0, cy - half) : min(h, cy + half),
                max(0, cx - half) : min(w, cx + half),
            ]
            patch = cv2.resize(patch, (patch_size, patch_size))
            return [patch] * num_patches

        # Sample random tissue locations
        for _ in range(num_patches * 3):  # Oversample to ensure we get enough
            if len(patches) >= num_patches:
                break

            # Random tissue point
            idx = np.random.randint(len(tissue_coords[0]))
            cy, cx = tissue_coords[0][idx], tissue_coords[1][idx]

            # Extract patch
            half = patch_size // 2
            y1, y2 = max(0, cy - half), min(h, cy + half)
            x1, x2 = max(0, cx - half), min(w, cx + half)

            patch = img[y1:y2, x1:x2]
            patch_mask = mask[y1:y2, x1:x2]

            # Check tissue ratio in patch
            tissue_ratio = patch_mask.sum() / (patch_mask.size * 255)
            if tissue_ratio > 0.3:  # At least 30% tissue
                if patch.shape[0] != patch_size or patch.shape[1] != patch_size:
                    patch = cv2.resize(patch, (patch_size, patch_size))
                patches.append(patch)

        # If not enough patches, duplicate
        while len(patches) < num_patches:
            if len(patches) > 0:
                patches.append(patches[np.random.randint(len(patches))])
            else:
                # Fallback: center crop
                cy, cx = h // 2, w // 2
                half = patch_size // 2
                patch = img[
                    max(0, cy - half) : min(h, cy + half),
                    max(0, cx - half) : min(w, cx + half),
                ]
                patch = cv2.resize(patch, (patch_size, patch_size))
                patches.append(patch)

        return patches[:num_patches]


In [ ]:
# ============================================================================
# Dataset
# ============================================================================
class PathologyDataset(Dataset):
    """Dataset optimized for histopathology images."""

    def __init__(
        self,
        data_dir: str,
        labels_df: Optional[pd.DataFrame] = None,
        transform: Optional[transforms.Compose] = None,
        img_size: int = 512,
        use_mask: bool = True,
        mask_mode: str = "channel",  # "multiply", "channel", or "attention"
        use_patches: bool = False,
        patch_size: int = 512,
        num_patches: int = 8,
        use_stain_norm: bool = True,
        is_test: bool = False,
        label_encoder: Optional[LabelEncoder] = None,
    ):
        self.data_dir = Path(data_dir)
        self.transform = transform
        self.img_size = img_size
        self.use_mask = use_mask
        self.mask_mode = mask_mode
        self.use_patches = use_patches
        self.patch_size = patch_size
        self.num_patches = num_patches
        self.use_stain_norm = use_stain_norm
        self.is_test = is_test
        self.label_encoder = label_encoder

        # Initialize helpers
        self.tissue_detector = TissueDetector()
        self.stain_normalizer = ReinhardNormalizer() if use_stain_norm else None

        if is_test:
            self.samples = self._get_test_samples()
            self.labels = None
        else:
            self.samples = [
                self._clean_sample_idx(str(idx))
                for idx in labels_df["sample_index"].tolist()
            ]
            self.labels = labels_df["label"].tolist()

            if self.label_encoder is None:
                self.label_encoder = LabelEncoder()
                self.label_encoder.fit(Config.CLASSES)
            self.encoded_labels = self.label_encoder.transform(self.labels)

    def _clean_sample_idx(self, sample_idx: str) -> str:
        """Clean sample index."""
        sample_idx = str(sample_idx)
        if sample_idx.startswith("img_"):
            sample_idx = sample_idx[4:]
        if sample_idx.endswith(".png"):
            sample_idx = sample_idx[:-4]
        return sample_idx

    def _get_test_samples(self) -> List[str]:
        """Get list of sample indices from test directory."""
        samples = []
        for f in sorted(self.data_dir.glob("img_*.png")):
            sample_idx = self._clean_sample_idx(f.stem)
            samples.append(sample_idx)
        return samples

    def __len__(self) -> int:
        return len(self.samples)

    def _load_and_preprocess(
        self, sample_idx: str
    ) -> Tuple[np.ndarray, Optional[np.ndarray]]:
        """Load and preprocess image with tissue detection. Returns (image, mask)."""
        img_path = self.data_dir / f"img_{sample_idx}.png"
        img = np.array(Image.open(img_path).convert("RGB"))

        # Load mask
        mask = None
        if self.use_mask:
            mask_path = self.data_dir / f"mask_{sample_idx}.png"
            if mask_path.exists():
                mask = np.array(Image.open(mask_path).convert("L"))
            else:
                # Auto-detect tissue if no mask provided
                mask = self.tissue_detector.detect_tissue_mask(img)

        # Crop to tissue region (crop both image and mask together)
        if mask is not None:
            x_min, y_min, x_max, y_max = self.tissue_detector.get_tissue_bbox(mask)
            img = img[y_min:y_max, x_min:x_max]
            mask = mask[y_min:y_max, x_min:x_max]

        # Stain normalization (only on RGB)
        if self.use_stain_norm and self.stain_normalizer is not None:
            try:
                img = self.stain_normalizer.normalize(img)
            except:
                pass

        # Resize both image and mask
        img = cv2.resize(img, (self.img_size, self.img_size))
        if mask is not None:
            mask = cv2.resize(mask, (self.img_size, self.img_size))

        return img, mask

    def _load_patches(
        self, sample_idx: str
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        """Load image as patches. Returns (image_patches, mask_patches)."""
        img_path = self.data_dir / f"img_{sample_idx}.png"
        img = np.array(Image.open(img_path).convert("RGB"))

        # Load mask
        mask = None
        if self.use_mask:
            mask_path = self.data_dir / f"mask_{sample_idx}.png"
            if mask_path.exists():
                mask = np.array(Image.open(mask_path).convert("L"))

        # Get patch coordinates
        h, w = img.shape[:2]
        tissue_coords = (
            np.where(mask > 0)
            if mask is not None
            else (np.array([h // 2]), np.array([w // 2]))
        )

        img_patches = []
        mask_patches = []

        for _ in range(self.num_patches * 3):
            if len(img_patches) >= self.num_patches:
                break

            if len(tissue_coords[0]) > 0:
                idx = np.random.randint(len(tissue_coords[0]))
                cy, cx = tissue_coords[0][idx], tissue_coords[1][idx]
            else:
                cy, cx = h // 2, w // 2

            half = self.patch_size // 2
            y1, y2 = max(0, cy - half), min(h, cy + half)
            x1, x2 = max(0, cx - half), min(w, cx + half)

            img_patch = img[y1:y2, x1:x2]
            mask_patch = (
                mask[y1:y2, x1:x2]
                if mask is not None
                else np.ones((y2 - y1, x2 - x1), dtype=np.uint8) * 255
            )

            tissue_ratio = (
                mask_patch.sum() / (mask_patch.size * 255) if mask is not None else 1.0
            )
            if tissue_ratio > 0.3:
                img_patch = cv2.resize(img_patch, (self.patch_size, self.patch_size))
                mask_patch = cv2.resize(mask_patch, (self.patch_size, self.patch_size))

                if self.use_stain_norm and self.stain_normalizer:
                    try:
                        img_patch = self.stain_normalizer.normalize(img_patch)
                    except:
                        pass

                img_patches.append(img_patch)
                mask_patches.append(mask_patch)

        # Ensure we have enough patches
        while len(img_patches) < self.num_patches:
            if len(img_patches) > 0:
                idx = np.random.randint(len(img_patches))
                img_patches.append(img_patches[idx])
                mask_patches.append(mask_patches[idx])
            else:
                # Fallback
                img_patch = cv2.resize(img, (self.patch_size, self.patch_size))
                mask_patch = cv2.resize(
                    mask if mask is not None else np.ones_like(img[:, :, 0]) * 255,
                    (self.patch_size, self.patch_size),
                )
                img_patches.append(img_patch)
                mask_patches.append(mask_patch)

        return img_patches[: self.num_patches], mask_patches[: self.num_patches]

    def _apply_transform(
        self, img: np.ndarray, mask: Optional[np.ndarray] = None
    ) -> torch.Tensor:
        """Apply transforms and combine image with mask based on mask_mode."""

        # Convert to PIL for transforms
        img_pil = Image.fromarray(img)

        if self.mask_mode == "channel" and mask is not None:
            # Apply transform to RGB image
            if self.transform:
                # We need to handle this carefully - transform RGB, then add mask channel
                # For now, apply basic transforms manually to keep mask aligned
                img_tensor = self.transform(img_pil)  # [3, H, W]
            else:
                img_tensor = transforms.ToTensor()(img_pil)

            # Normalize mask to 0-1 and resize to match transformed image size
            mask_normalized = mask.astype(np.float32) / 255.0
            mask_tensor = torch.from_numpy(mask_normalized).unsqueeze(0)  # [1, H, W]

            # Resize mask if needed
            if mask_tensor.shape[1:] != img_tensor.shape[1:]:
                mask_tensor = F.interpolate(
                    mask_tensor.unsqueeze(0), size=img_tensor.shape[1:], mode="nearest"
                ).squeeze(0)

            # Concatenate RGB + Mask -> 4 channels
            combined = torch.cat([img_tensor, mask_tensor], dim=0)  # [4, H, W]
            return combined

        elif self.mask_mode == "multiply" and mask is not None:
            # Apply mask by multiplication
            mask_normalized = mask.astype(np.float32) / 255.0
            mask_3d = np.stack([mask_normalized] * 3, axis=-1)
            img_masked = (img * mask_3d).astype(np.uint8)
            img_pil = Image.fromarray(img_masked)

            if self.transform:
                return self.transform(img_pil)
            else:
                return transforms.ToTensor()(img_pil)

        else:
            # No mask or attention mode
            if self.transform:
                return self.transform(img_pil)
            else:
                return transforms.ToTensor()(img_pil)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, ...]:
        sample_idx = self.samples[idx]

        if self.use_patches:
            img_patches, mask_patches = self._load_patches(sample_idx)

            transformed_patches = []
            for img_patch, mask_patch in zip(img_patches, mask_patches):
                patch_tensor = self._apply_transform(img_patch, mask_patch)
                transformed_patches.append(patch_tensor)

            img_tensor = torch.stack(transformed_patches)  # [num_patches, C, H, W]
        else:
            img, mask = self._load_and_preprocess(sample_idx)
            img_tensor = self._apply_transform(img, mask)

        if self.is_test:
            return img_tensor, sample_idx
        else:
            label = self.encoded_labels[idx]
            return img_tensor, torch.tensor(label, dtype=torch.long)


In [ ]:
# ============================================================================
# Data Module
# ============================================================================
class PathologyDataModule(LightningDataModule):
    """DataModule for pathology images."""

    def __init__(self, config: Config = Config()):
        super().__init__()
        self.config = config
        self.label_encoder = LabelEncoder()
        self.label_encoder.fit(config.CLASSES)

        # Transforms
        self.train_transform = transforms.Compose(
            [
                transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=90),
                transforms.ColorJitter(
                    brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05
                ),
                transforms.RandomAffine(
                    degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)
                ),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

        self.val_transform = transforms.Compose(
            [
                transforms.Resize((config.IMG_SIZE, config.IMG_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        )

    def setup(self, stage: Optional[str] = None):
        if stage == "fit" or stage is None:
            labels_df = pd.read_csv(self.config.TRAIN_LABELS_PATH)

            def clean_idx(idx):
                idx = str(idx)
                if idx.startswith("img_"):
                    idx = idx[4:]
                if idx.endswith(".png"):
                    idx = idx[:-4]
                return idx

            labels_df["sample_index"] = labels_df["sample_index"].apply(clean_idx)

            train_df, val_df = train_test_split(
                labels_df,
                test_size=self.config.VAL_SPLIT,
                stratify=labels_df["label"],
                random_state=self.config.RANDOM_SEED,
            )

            self.train_dataset = PathologyDataset(
                data_dir=self.config.TRAIN_DATA_DIR,
                labels_df=train_df,
                transform=self.train_transform,
                img_size=self.config.IMG_SIZE,
                use_mask=self.config.USE_MASK,
                mask_mode=self.config.MASK_MODE,
                use_patches=self.config.USE_PATCHES,
                patch_size=self.config.PATCH_SIZE,
                num_patches=self.config.NUM_PATCHES,
                use_stain_norm=self.config.USE_STAIN_NORMALIZATION,
                label_encoder=self.label_encoder,
            )

            self.val_dataset = PathologyDataset(
                data_dir=self.config.TRAIN_DATA_DIR,
                labels_df=val_df,
                transform=self.val_transform,
                img_size=self.config.IMG_SIZE,
                use_mask=self.config.USE_MASK,
                mask_mode=self.config.MASK_MODE,
                use_patches=self.config.USE_PATCHES,
                patch_size=self.config.PATCH_SIZE,
                num_patches=self.config.NUM_PATCHES,
                use_stain_norm=self.config.USE_STAIN_NORMALIZATION,
                label_encoder=self.label_encoder,
            )

            # Calculate class weights for balanced sampling
            class_counts = train_df["label"].value_counts()
            weights = 1.0 / class_counts[train_df["label"].values].values
            self.sample_weights = torch.DoubleTensor(weights)

        if stage == "test" or stage == "predict" or stage is None:
            self.test_dataset = PathologyDataset(
                data_dir=self.config.TEST_DATA_DIR,
                transform=self.val_transform,
                img_size=self.config.IMG_SIZE,
                use_mask=self.config.USE_MASK,
                mask_mode=self.config.MASK_MODE,
                use_patches=self.config.USE_PATCHES,
                patch_size=self.config.PATCH_SIZE,
                num_patches=self.config.NUM_PATCHES,
                use_stain_norm=self.config.USE_STAIN_NORMALIZATION,
                is_test=True,
                label_encoder=self.label_encoder,
            )

    def train_dataloader(self) -> DataLoader:
        sampler = WeightedRandomSampler(
            self.sample_weights, len(self.sample_weights), replacement=True
        )

        return DataLoader(
            self.train_dataset,
            batch_size=self.config.BATCH_SIZE,
            sampler=sampler,
            num_workers=self.config.NUM_WORKERS,
            pin_memory=True,
            drop_last=True,
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_dataset,
            batch_size=self.config.BATCH_SIZE,
            shuffle=False,
            num_workers=self.config.NUM_WORKERS,
            pin_memory=True,
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.test_dataset,
            batch_size=self.config.BATCH_SIZE,
            shuffle=False,
            num_workers=self.config.NUM_WORKERS,
            pin_memory=True,
        )

    def predict_dataloader(self) -> DataLoader:
        return self.test_dataloader()

In [ ]:
# ============================================================================
# Model with Multi-Instance Learning (MIL) for patches
# ============================================================================
class AttentionMIL(nn.Module):
    """Attention-based Multiple Instance Learning pooling."""

    def __init__(self, feature_dim: int, hidden_dim: int = 256):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim), nn.Tanh(), nn.Linear(hidden_dim, 1)
        )

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
            x: [batch, num_instances, features]
        Returns:
            pooled: [batch, features]
            attention_weights: [batch, num_instances]
        """
        # Compute attention scores
        attn_scores = self.attention(x)  # [batch, num_instances, 1]
        attn_weights = F.softmax(attn_scores, dim=1)  # [batch, num_instances, 1]

        # Weighted sum
        pooled = torch.sum(x * attn_weights, dim=1)  # [batch, features]

        return pooled, attn_weights.squeeze(-1)


class PathologyClassifier(LightningModule):
    """Classifier optimized for histopathology with optional MIL and 4-channel input."""

    def __init__(
        self,
        num_classes: int = Config.NUM_CLASSES,
        model_name: str = Config.MODEL_NAME,
        pretrained: bool = Config.PRETRAINED,
        learning_rate: float = Config.LEARNING_RATE,
        weight_decay: float = Config.WEIGHT_DECAY,
        class_names: List[str] = Config.CLASSES,
        use_patches: bool = Config.USE_PATCHES,
        num_patches: int = Config.NUM_PATCHES,
        in_channels: int = Config.IN_CHANNELS,  # 3 for RGB, 4 for RGB+Mask
    ):
        super().__init__()
        self.save_hyperparameters()

        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.class_names = class_names
        self.use_patches = use_patches
        self.num_patches = num_patches
        self.in_channels = in_channels

        # Create backbone
        self.backbone, self.feature_dim = self._create_backbone(
            model_name, pretrained, in_channels
        )

        # MIL attention pooling (for patch-based)
        if use_patches:
            self.mil_attention = AttentionMIL(self.feature_dim)

        # Classification head
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.feature_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes),
        )

        # Loss function
        self.criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

        # Metrics storage
        self.validation_step_outputs = []

    def _modify_first_conv(
        self, conv: nn.Conv2d, in_channels: int, pretrained: bool
    ) -> nn.Conv2d:
        """Modify the first conv layer to accept different number of input channels."""
        if in_channels == 3:
            return conv

        # Create new conv layer with desired input channels
        new_conv = nn.Conv2d(
            in_channels=in_channels,
            out_channels=conv.out_channels,
            kernel_size=conv.kernel_size,
            stride=conv.stride,
            padding=conv.padding,
            bias=conv.bias is not None,
        )

        if pretrained:
            # Initialize weights: copy RGB weights and initialize new channels
            with torch.no_grad():
                # Copy weights for first 3 channels (RGB)
                new_conv.weight[:, :3, :, :] = conv.weight.clone()

                # Initialize additional channels (e.g., mask channel)
                # Option 1: Initialize with mean of RGB weights
                if in_channels > 3:
                    mean_weight = conv.weight.mean(dim=1, keepdim=True)
                    for i in range(3, in_channels):
                        new_conv.weight[:, i : i + 1, :, :] = mean_weight.clone()

                # Copy bias if exists
                if conv.bias is not None:
                    new_conv.bias = nn.Parameter(conv.bias.clone())

        return new_conv

    def _create_backbone(
        self, model_name: str, pretrained: bool, in_channels: int
    ) -> Tuple[nn.Module, int]:
        """Create backbone model with support for custom input channels."""

        if model_name == "resnet50":
            weights = models.ResNet50_Weights.IMAGENET1K_V2 if pretrained else None
            model = models.resnet50(weights=weights)
            feature_dim = 2048

            # Modify first conv if needed
            if in_channels != 3:
                model.conv1 = self._modify_first_conv(
                    model.conv1, in_channels, pretrained
                )

            backbone = nn.Sequential(*list(model.children())[:-1], nn.Flatten())

        elif model_name == "resnet101":
            weights = models.ResNet101_Weights.IMAGENET1K_V2 if pretrained else None
            model = models.resnet101(weights=weights)
            feature_dim = 2048

            if in_channels != 3:
                model.conv1 = self._modify_first_conv(
                    model.conv1, in_channels, pretrained
                )

            backbone = nn.Sequential(*list(model.children())[:-1], nn.Flatten())

        elif model_name == "efficientnet_b3":
            weights = (
                models.EfficientNet_B3_Weights.IMAGENET1K_V1 if pretrained else None
            )
            model = models.efficientnet_b3(weights=weights)
            feature_dim = 1536

            # EfficientNet first conv is in features[0][0]
            if in_channels != 3:
                model.features[0][0] = self._modify_first_conv(
                    model.features[0][0], in_channels, pretrained
                )

            backbone = nn.Sequential(
                model.features, nn.AdaptiveAvgPool2d(1), nn.Flatten()
            )

        elif model_name == "efficientnet_b4":
            weights = (
                models.EfficientNet_B4_Weights.IMAGENET1K_V1 if pretrained else None
            )
            model = models.efficientnet_b4(weights=weights)
            feature_dim = 1792

            if in_channels != 3:
                model.features[0][0] = self._modify_first_conv(
                    model.features[0][0], in_channels, pretrained
                )

            backbone = nn.Sequential(
                model.features, nn.AdaptiveAvgPool2d(1), nn.Flatten()
            )

        elif model_name == "convnext_small":
            weights = (
                models.ConvNeXt_Small_Weights.IMAGENET1K_V1 if pretrained else None
            )
            model = models.convnext_small(weights=weights)
            feature_dim = 768

            # ConvNeXt first conv is in features[0][0]
            if in_channels != 3:
                model.features[0][0] = self._modify_first_conv(
                    model.features[0][0], in_channels, pretrained
                )

            backbone = nn.Sequential(
                model.features, nn.AdaptiveAvgPool2d(1), nn.Flatten()
            )

        elif model_name == "densenet121":
            weights = models.DenseNet121_Weights.IMAGENET1K_V1 if pretrained else None
            model = models.densenet121(weights=weights)
            feature_dim = 1024

            # DenseNet first conv is in features.conv0
            if in_channels != 3:
                model.features.conv0 = self._modify_first_conv(
                    model.features.conv0, in_channels, pretrained
                )

            backbone = nn.Sequential(
                model.features,
                nn.ReLU(inplace=True),
                nn.AdaptiveAvgPool2d(1),
                nn.Flatten(),
            )

        else:
            raise ValueError(f"Unsupported model: {model_name}")

        return backbone, feature_dim

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if self.use_patches:
            # x shape: [batch, num_patches, C, H, W]
            batch_size, num_patches, C, H, W = x.shape

            # Flatten batch and patches
            x = x.view(batch_size * num_patches, C, H, W)

            # Extract features
            features = self.backbone(x)  # [batch*num_patches, feature_dim]

            # Reshape back
            features = features.view(
                batch_size, num_patches, -1
            )  # [batch, num_patches, feature_dim]

            # MIL attention pooling
            pooled, _ = self.mil_attention(features)  # [batch, feature_dim]

            # Classify
            logits = self.classifier(pooled)
        else:
            # x shape: [batch, C, H, W]
            features = self.backbone(x)
            logits = self.classifier(features)

        return logits

    def training_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int
    ) -> torch.Tensor:
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(
        self, batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int
    ) -> dict:
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)

        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()

        self.log("val_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_step=False, on_epoch=True, prog_bar=True)

        self.validation_step_outputs.append({"preds": preds, "labels": labels})

        return {"val_loss": loss, "val_acc": acc}

    def on_validation_epoch_end(self):
        all_preds = torch.cat([x["preds"] for x in self.validation_step_outputs])
        all_labels = torch.cat([x["labels"] for x in self.validation_step_outputs])

        for i, class_name in enumerate(self.class_names):
            mask = all_labels == i
            if mask.sum() > 0:
                class_acc = (all_preds[mask] == all_labels[mask]).float().mean()
                self.log(f"val_acc_{class_name}", class_acc, on_epoch=True)

        self.validation_step_outputs.clear()

    def predict_step(
        self, batch: Tuple[torch.Tensor, str], batch_idx: int
    ) -> Tuple[torch.Tensor, List[str]]:
        images, sample_indices = batch
        logits = self(images)
        preds = torch.argmax(logits, dim=1)
        return preds, sample_indices

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )

        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=2, eta_min=1e-7
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "interval": "epoch"},
        }

In [ ]:
# ============================================================================
# Training and Inference
# ============================================================================
def train_model(
    config: Config = Config(),
) -> Tuple[PathologyClassifier, PathologyDataModule, Trainer]:
    """Train the pathology classifier."""

    pl.seed_everything(config.RANDOM_SEED)

    data_module = PathologyDataModule(config)

    model = PathologyClassifier(
        num_classes=config.NUM_CLASSES,
        model_name=config.MODEL_NAME,
        pretrained=config.PRETRAINED,
        learning_rate=config.LEARNING_RATE,
        weight_decay=config.WEIGHT_DECAY,
        class_names=config.CLASSES,
        use_patches=config.USE_PATCHES,
        num_patches=config.NUM_PATCHES,
        in_channels=config.IN_CHANNELS,
    )

    checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="pathology-{epoch:02d}-{val_acc:.4f}",
        monitor="val_acc",
        mode="max",
        save_top_k=3,
        save_last=True,
    )

    early_stopping = EarlyStopping(monitor="val_acc", patience=15, mode="max")

    lr_monitor = LearningRateMonitor(logging_interval="epoch")

    logger = TensorBoardLogger("logs", name="pathology")

    trainer = Trainer(
        max_epochs=config.MAX_EPOCHS,
        accelerator="auto",
        devices="auto",
        precision="16-mixed",
        callbacks=[checkpoint_callback, early_stopping, lr_monitor],
        logger=logger,
        gradient_clip_val=1.0,
        accumulate_grad_batches=16,
        deterministic=True,
    )

    trainer.fit(model, datamodule=data_module)

    return model, data_module, trainer


def predict_and_save(
    model: PathologyClassifier,
    data_module: PathologyDataModule,
    trainer: Trainer,
    output_path: str = Config.OUTPUT_PATH,
) -> pd.DataFrame:
    """Run inference and save predictions."""

    data_module.setup(stage="predict")
    predictions = trainer.predict(model, datamodule=data_module)

    all_preds = []
    all_indices = []

    for preds, indices in predictions:
        all_preds.extend(preds.cpu().numpy())
        all_indices.extend(indices)

    label_encoder = data_module.label_encoder
    predicted_labels = label_encoder.inverse_transform(all_preds)

    results_df = pd.DataFrame({"sample_index": all_indices, "label": predicted_labels})

    results_df = results_df.sort_values("sample_index").reset_index(drop=True)
    results_df.to_csv(output_path, index=False)

    print(f"Predictions saved to: {output_path}")

    return results_df

In [ ]:
config = Config()

print("=" * 60)
print("Pathology-Optimized Molecular Subtype Classification")
print("=" * 60)
print(f"Model: {config.MODEL_NAME}")
print(f"Image Size: {config.IMG_SIZE}")
print(
    f"Input Channels: {config.IN_CHANNELS} ({'RGB + Mask' if config.IN_CHANNELS == 4 else 'RGB'})"
)
print(f"Mask Mode: {config.MASK_MODE}")
print(f"Use Patches: {config.USE_PATCHES}")
if config.USE_PATCHES:
    print(f"  - Patch Size: {config.PATCH_SIZE}")
    print(f"  - Num Patches: {config.NUM_PATCHES}")
print(f"Stain Normalization: {config.USE_STAIN_NORMALIZATION}")
print(f"Batch Size: {config.BATCH_SIZE}")
print("=" * 60)

print("\n[1/2] Training model...")
model, data_module, trainer = train_model(config)

In [ ]:
print("\n[2/2] Running inference on test set...")
results_df = predict_and_save(model, data_module, trainer, config.OUTPUT_PATH)

print("\n" + "=" * 60)
print("Prediction Summary")
print("=" * 60)
print(results_df["label"].value_counts())
print("\nDone!")

In [ ]:
results_df